In [370]:
import pandas as pd
import sqlite3
from Levenshtein import distance
from fuzzywuzzy import fuzz

# connect to the SQLite database and read the data into a pandas dataframe
conn = sqlite3.connect('data_files/spanish/words.db')
df = pd.read_sql('SELECT * FROM words', conn)

# close the database connection
conn.close()

In [371]:
df.columns

Index(['word', 'type', 'translation', 'example', 'example_translation',
       'russian', 'appear', 'trial_d', 'trial_r', 'success', 'weight',
       'word_index'],
      dtype='object')

In [ ]:
df = df.drop(columns=['difficulty', 'wd'])

In [372]:
columns = list(df.columns)

df['Length_word'] = df['word'].str.len()
df['translation'] = df['translation'].str.split(',').str[0]
df['Length_translation'] = df['translation'].str.len()
# Calculate Levenshtein Distance between word and translation
df['Similarity'] = df.apply(lambda row: fuzz.ratio(row['word'], row['translation']), axis=1)
    
# dividing into quantiles
df['d1'] = pd.qcut(df['Length_word'], q=4, labels=[0, 1, 2, 3])
df['d1'] = df['d1'].astype(int)
df['d2'] = pd.qcut(df['Length_translation'], q=3, labels=[0, 1, 2])
df['d2'] = df['d2'].astype(int)
df['d3'] = pd.qcut(df['Similarity'], q=5, labels=[4, 3, 2, 1, 0])
df['d3'] = df['d3'].astype(int)
df['d4'] = pd.cut(df['weight'], bins=3, labels=[1, 2, 3])
df['d4'] = df['d4'].astype(int)

df['difficulty'] = df['d1'] + df['d2'] + df['d3']
df['wd'] = df['d1'] + df['d2'] + df['d3'] + df['d4']

columns = columns + ['difficulty', 'wd']

df = df.loc[:, columns]

In [373]:
df['wd'].unique()

array([ 5,  6,  4,  2,  7,  8,  3,  9, 10, 11])

In [374]:
def xlstosql(df):
    df_name = df.name
    # connect to the SQLite database
    conn = sqlite3.connect(f'data_files/spanish/{df_name}.db')
    # insert the data from the dataframe into the database table
    df.to_sql(f'{df_name}', conn, if_exists='replace', index=False)
    # close the database connection
    conn.close()

In [375]:
df.name = 'words'
xlstosql(df)

In [213]:
dif_df = df.loc[:, ['word_index', 'difficulty']]

In [366]:
conn = sqlite3.connect('data_files/spanish/words.db')
cursor = conn.cursor()

cursor.execute("ALTER TABLE words ADD COLUMN word_index INTEGER")

cursor.execute("SELECT rowid FROM words")
rows = cursor.fetchall()

for i, row in enumerate(rows, start=1):
    cursor.execute("UPDATE words SET word_index = ? WHERE rowid = ?", (i, row[0]))
conn.commit()

# Сохранение изменений и закрытие соединения
conn.close()

In [237]:
def difficulty(df, level='standard'):
    
    columns = list(df.columns)
    
    df['Length_word'] = df['word'].str.len()
    df['translation'] = df['translation'].str.split(',').str[0]
    df['Length_translation'] = df['translation'].str.len()
    # Calculate Levenshtein Distance between word and translation
    df['Similarity'] = df.apply(lambda row: fuzz.ratio(row['word'], row['translation']), axis=1)
    
    # dividing into quantiles
    df['d1'] = pd.qcut(df['Length_word'], q=4, labels=[0, 1, 2, 3])
    df['d1'] = df['d1'].astype(int)
    df['d2'] = pd.qcut(df['Length_translation'], q=3, labels=[0, 1, 2])
    df['d2'] = df['d2'].astype(int)
    df['d3'] = pd.qcut(df['Similarity'], q=5, labels=[4, 3, 2, 1, 0])
    df['d3'] = df['d3'].astype(int)
    df['d4'] = pd.qcut(df['weight'], bins=3, labels=[1, 2, 3], duplicates='drop')
    df['d4'] = df['d4'].astype(int)
    
    df['difficulty'] = df['d1'] + df['d2'] + df['d3'] + df['d4']
    
    
    
    df = df.loc[:, columns]
    return df

In [198]:
# Calculate word lengths and divide into three bins
#df['Difficulty'] = pd.cut(df['Length'], bins=3, labels=['Easy', 'Medium', 'Hard'])

In [228]:
test_list = []

for i in range(0,101):
    test_list.append(i)

In [229]:
tt=pd.DataFrame()
tt['count'] = test_list


In [230]:
tt['dif'] = pd.cut(test_list, bins=3, labels=['Easy', 'Medium', 'Hard'])

In [266]:
writer = pd.ExcelWriter('data_files/tdddt.xlsx')
df.to_excel(writer, sheet_name='update')
writer.close()

In [381]:
conn2 = sqlite3.connect('data_files/lessons.db')
lesson = pd.read_sql('SELECT * FROM lessons', conn2)
conn2.close()

In [382]:
lesson.columns

Index(['lesson', 'start', 'inter', 'finish', 'known', 'points', 'length',
       'time', 'list_of_words', 'r', 'new_col'],
      dtype='object')

In [139]:
lesson_df = lesson[lesson['known'] != 25]
lesson_df = lesson_df.loc[:, ['r', 'list_of_words', 'points']]

In [140]:
# Create a sample difficulty DataFrame
difficulty_df = df.loc[:,['word', 'difficulty']]

# Split 'list of words' column and retrieve difficulties
lesson_df['words'] = lesson_df['list_of_words'].str.split(';')
lesson_df['words'] = lesson_df['words'].apply(lambda x: [word.strip() for word in x])
lesson_df['dif'] = lesson_df['words'].apply(lambda x: sum(difficulty_df[difficulty_df['word'].isin(x)]['difficulty']))

In [141]:
# Remove the intermediate 'words' column
lesson_df.drop('words', axis=1, inplace=True)

In [142]:
writer = pd.ExcelWriter('data_files/test_check.xlsx')
lesson_df.to_excel(writer, sheet_name='update')
writer.close()

In [143]:
# Add a unique key column based on index
df['key'] = df.index

# Define the desired distribution of words by difficulty
difficulty_distribution_easy = {1: 4, 2: 4, 3: 4, 4: 4, 5: 4, 6:2, 7:1, 8:1 , 9:1, 10:0}
difficulty_distribution = {1: 2, 2: 2, 3: 3, 4: 3, 5: 3, 6: 3, 7:3, 8:2 ,9:2, 10:2 }
difficulty_distribution_hard = {1: 0, 2: 1, 3: 1, 4: 1, 5: 2, 6: 4, 7:4, 8:4 ,9:4, 10:4 }
difficulty_distribution_hardest = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 5, 7:5, 8:5 ,9:5, 10:5 }

# Sample words from each difficulty level based on difficulty and weight
selected_words = []
for difficulty, count in difficulty_distribution.items():
    words_subset = df[(df['wb'] == difficulty)]
    words_subset = words_subset.sample(n=count, weights=words_subset['weight'])
    selected_words.append(words_subset)
    
# Concatenate the selected words into a new DataFrame
new_words_df = pd.concat(selected_words)

writer = pd.ExcelWriter('data_files/new.xlsx')
new_words_df.to_excel(writer, sheet_name='update')
writer.close()

In [122]:
# Reset the index of the new DataFrame
new_words_df.reset_index(drop=True, inplace=True)

# Update the parameters of the selected words
new_words_df['weight'] = new_words_df['weight'] * 2  # Update the weight parameter as an example

# Merge the updated words with the original DataFrame based on the key column
df = pd.merge(df, new_words_df, on='key', how='left')

# Fill missing values in the original DataFrame with the updated values
df['difficulty_y'].fillna(df['difficulty_x'], inplace=True)
df['weight_y'].fillna(df['weight_x'], inplace=True)

# Drop redundant columns from the merged DataFrame
df.drop(['difficulty_x', 'weight_x', 'key'], axis=1, inplace=True)
df.rename(columns={'difficulty_y': 'difficulty', 'weight_y': 'weight'}, inplace=True)

writer = pd.ExcelWriter('data_files/final.xlsx')
df.to_excel(writer, sheet_name='update')
writer.close()

In [ ]:
', '.join(list(df.columns))

In [ ]:
import sqlite3
import random

difficulty_distribution = {1: 1, 2: 1, 3: 2, 4: 2, 5: 3, 6: 3, 7: 3, 8: 3, 9: 3, 10: 2, 11: 1, 12: 1}

def sql_text(dffty, limit):
    text = f"""
    SELECT word, type, translation, russian, example, example_translation, appear, trial_d, trial_r, success, weight, word_index, difficulty, wd
    FROM words
    WHERE wd = {dffty}
    ORDER BY weight DESC, RANDOM()
    LIMIT {limit};
    """
    return text

def loadData(source, final='no'):
    # connect to the SQLite database and read the data into a pandas dataframe
    conn = sqlite3.connect('data_files/words.db')
    
    
    if source == 'word' and final == 'no':
        cursor = conn.cursor()
        # Sample words from each difficulty level based on difficulty and weight
        selected_words = []
        for difficulty, count in difficulty_distribution.items():
            # Retrieve 5 easy words
            words_query = sql_text(difficulty, count)
            cursor.execute(words_query)
            selected_words = selected_words + cursor.fetchall()
        return selected_words
    else:
        df = pd.read_sql(f'SELECT * FROM {source}s', conn)
        df = df.loc[:, f'{source}':]
        
    # close the database connection
    conn.close()
    

combined_data = loadData('word')

    
class Words(object):

    """The class for all words from dictionary. Each word a type, a translation, an example,
    a translation of an example, a translation to Russian, also additional parameters which would be created after
    the first run (quantity of appearances in lessons, trials of direct and indirect translation,
    success attempts, also weight, more weight - more often words appear"""

    def __init__(self, word, typ, translation, russian, example, example_translation, appear, trial_d, trial_r, success,
                 weight, word_index, difficulty, wd):
        self.word = word
        self.typ = typ
        self.translation = translation
        self.example = example
        self.example_translation = example_translation
        self.russian = russian
        self.appear = appear
        self.trial_d = trial_d
        self.trial_r = trial_r
        self.success = success
        self.weight = weight
        self.word_index = word_index
        self.difficulty = difficulty
        self.wd = wd


# Create Word objects from the combined data
word_objects = []
for row in combined_data:
    word = Words(*row)
    word_objects.append(word)

# Print the created Word objects
for word in word_objects:
    print(f"Word: {word.word}, Translation: {word.translation}, Weight: {word.weight}, Difficulty: {word.wd}")


In [ ]:
def sql_text(dffty, limit):
    text = f"""
    SELECT word, type, translation, russian, example, example_translation, appear, trial_d, trial_r, success, weight, word_index, difficulty, wd
    FROM words
    WHERE wd = {dffty}
    ORDER BY weight DESC, RANDOM()
    LIMIT {limit};
    """
    return text


# connect to the SQLite database and read the data into a pandas dataframe
conn = sqlite3.connect('data_files/words.db')
cursor = conn.cursor()

# Retrieve 5 easy words
easy_words_query = sql_text(2, 15)
cursor.execute(easy_words_query)
easy_words = cursor.fetchall()

# Retrieve 5 standard words
standard_words_query = sql_text(6, 20)
cursor.execute(standard_words_query)
standard_words = cursor.fetchall()

# Retrieve 5 hard words
hard_words_query = sql_text(10, 5)
cursor.execute(hard_words_query)
hard_words = cursor.fetchall()

# Close the database connection
conn.close()

# Print the retrieved words

print("\nEasy Words:")
for row in easy_words:
    print(row)

print("\nStandard Words:")
for row in standard_words:
    print(row)

print("\nHard Words:")
for row in hard_words:
    print(row)

In [292]:

text = """
    SELECT weight, word_index, wd
    FROM words
    """

# connect to the SQLite database and read the data into a pandas dataframe
conn = sqlite3.connect('data_files/words.db')
cursor = conn.cursor()

# Retrieve 5 easy words
easy_words_query = text
cursor.execute(easy_words_query)
easy_words = cursor.fetchall()

# Close the database connection
conn.close()

In [295]:
easy_words[1]

(0.0, 2, 3)

In [306]:
conn = sqlite3.connect('data_files/words.db')
cursor = conn.cursor()

v1 = 100
v2 = 100
v3 = 100
v4 = 100
v5 = 100

cursor.execute("UPDATE words SET appear = ?, trial_d = ?, trial_r = ?, success = ?, wd = ?  WHERE word_index = ?",
            (v1, v2, v3, v4, v5, 1))

    # Commit the changes and close the connection
conn.commit()
conn.close()

In [311]:
text = """

UPDATE words
SET weight = 
             CASE
                WHEN trial_d = 0 AND trial_r = 0 THEN ROUND(100, 2)
                ELSE ROUND((100 - (success / (trial_d + trial_r)) * 100), 2)
             END;
"""

conn = sqlite3.connect('data_files/words.db')
cursor = conn.cursor()

cursor.execute(text)
conn.commit()
conn.close()

In [329]:
text = """

UPDATE words
SET weight = 
    CASE
        WHEN trial_d = 0 AND trial_r = 0 THEN ROUND(100, 2)
        ELSE ROUND((100 - success * 100 /(trial_d + trial_r)), 2)
    END;
"""

conn = sqlite3.connect('data_files/words.db')
cursor = conn.cursor()

cursor.execute(text)
conn.commit()
conn.close()

In [332]:
conn = sqlite3.connect('data_files/lessons.db')
cursor = conn.cursor()

cursor.execute("SELECT lesson FROM lessons WHERE lesson IS NOT NULL ORDER BY lesson DESC LIMIT 1")
last_lesson = cursor.fetchone()[0]
l1 = last_lesson + 1
l2 = 0
l3 = 0
l4 = 0
l5 = 0
l6 = 0
l7 = 0
l8 = 0
l9 = 0
l10 = 0
    
cols = 'lesson, start, inter, finish, known, points, length, time, list_of_words, r'

cursor.execute(f"INSERT INTO lessons ({cols}) VALUES (?,?,?,?,?,?,?,?,?,?)", (l1,l2,l3,l4,l5,l6,l7,l8,l9,l10))

conn.commit()
conn.close()

In [ ]:
import sqlite3

# Соединение с базой данных SQLite3
conn = sqlite3.connect('data_files/lessons.db')
cursor = conn.cursor()

# Выполнение запроса SELECT для получения уникальных значений из столбца
cursor.execute("SELECT DISTINCT r FROM lessons")
results = cursor.fetchall()

# Вывод уникальных значений в консоль
for row in results:
    print(row[0])

# Закрытие соединения с базой данных
conn.close()

In [387]:
conn1 = sqlite3.connect('data_files/words.db')
cursor1 = conn1.cursor()

cursor1.execute('SELECT word_index, word FROM words')
results = cursor1.fetchall()

conn1.close()


# Соединение с базой данных SQLite3
conn = sqlite3.connect('data_files/lessons.db')
cursor = conn.cursor()

# Выполнение запроса SELECT для получения уникальных значений из столбца
cursor.execute("SELECT list_of_words FROM lessons")
lessons_x = cursor.fetchall()


conn.close()


for k, les in enumerate(lessons_x):
    for row in les:
        lesson_words = row.split(";")
        for i, cw in enumerate(lesson_words):
            lesson_words[i] = cw.strip()

            for res in results:
                if res[1] == lesson_words[i]:
                    lesson_words[i] = res[0]
    lessons_x[k] = lesson_words

25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
27
25
25
25
25
26
25
25
25
25
25
25
25
25
25
26
25
26
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
24
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
24
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25


In [390]:
import sqlite3

# Connection to the 'words.db' database
conn1 = sqlite3.connect('data_files/words.db')
cursor1 = conn1.cursor()

cursor1.execute('SELECT word_index, word FROM words')
results = cursor1.fetchall()

conn1.close()

# Connection to the 'lessons.db' database
conn = sqlite3.connect('data_files/lessons.db')
cursor = conn.cursor()

# Retrieve the list_of_words values from the 'lessons' table
cursor.execute("SELECT list_of_words FROM lessons")
lessons_x = cursor.fetchall()

# Process and update the values
for k, les in enumerate(lessons_x):
    for row in les:
        lesson_words = row.split(";")
        for i, cw in enumerate(lesson_words):
            lesson_words[i] = cw.strip()

            for res in results:
                if res[1] == lesson_words[i]:
                    lesson_words[i] = res[0]
        lesson_words = [str(word) for word in lesson_words if word]

    lessons_x[k] = ';'.join(lesson_words)

    # Update the 'list_of_words' column with the new value
    update_query = "UPDATE lessons SET list_of_words = ? WHERE rowid = ?"
    cursor.execute(update_query, (lessons_x[k], k + 1))
    conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

In [ ]:
# Соединение с базой данных SQLite3
conn = sqlite3.connect('data_files/lessons.db')
cursor = conn.cursor()

#cursor.execute("ALTER TABLE lessons ADD COLUMN new_col TEXT;")

# Выполнение запроса SELECT для получения уникальных значений из столбца
cursor.execute("UPDATE lessons SET ind = clmn;")


conn.close()

In [ ]:
conn1 = sqlite3.connect('data_files/words.db')
cursor1 = conn1.cursor()

cursor1.execute('SELECT word_index, word FROM words')
results = cursor1.fetchall()

conn1.close()

In [385]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data_files/lessons.db')
cursor = conn.cursor()

# Create a new table without the 'word' column
create_query = "CREATE TABLE new_table_name (lesson INTEGER, start TIMESTAMP, inter TIMESTAMP, finish TIMESTAMP, known INTEGER, points INTEGER, length INTEGER,time INTEGER, list_of_words TEXT, r INTEGER)"  
    

cursor.execute(create_query)

# Copy data from the old table to the new table
copy_query = "INSERT INTO new_table_name (lesson , start , inter , finish , known , points , length ,time , list_of_words , r ) SELECT lesson , start , inter , finish , known , points , length ,time , list_of_words , r  FROM lessons"
cursor.execute(copy_query)

# Drop the old table
drop_query = "DROP TABLE lessons"
cursor.execute(drop_query)

# Rename the new table to the original table name
rename_query = "ALTER TABLE new_table_name RENAME TO lessons"
cursor.execute(rename_query)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [383]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data_files/lessons.db')
cursor = conn.cursor()

# Define the table name
table_name = 'lessons'

# Retrieve column information from sqlite_master table
cursor.execute(f"PRAGMA table_info({table_name})")
columns = cursor.fetchall()

print(columns)

# Iterate over the columns and print the name and datatype
for column in columns:
    column_name = column[1]
    column_datatype = column[2]
    print(f"Column: {column_name} - Datatype: {column_datatype}")

# Close the connection
conn.close()

[(0, 'lesson', 'INTEGER', 0, None, 0), (1, 'start', 'TIMESTAMP', 0, None, 0), (2, 'inter', 'TIMESTAMP', 0, None, 0), (3, 'finish', 'TIMESTAMP', 0, None, 0), (4, 'known', 'INTEGER', 0, None, 0), (5, 'points', 'INTEGER', 0, None, 0), (6, 'length', 'INTEGER', 0, None, 0), (7, 'time', 'INTEGER', 0, None, 0), (8, 'list_of_words', 'TEXT', 0, None, 0), (9, 'r', 'INTEGER', 0, None, 0), (10, 'new_col', 'TEXT', 0, None, 0)]
Column: lesson - Datatype: INTEGER
Column: start - Datatype: TIMESTAMP
Column: inter - Datatype: TIMESTAMP
Column: finish - Datatype: TIMESTAMP
Column: known - Datatype: INTEGER
Column: points - Datatype: INTEGER
Column: length - Datatype: INTEGER
Column: time - Datatype: INTEGER
Column: list_of_words - Datatype: TEXT
Column: r - Datatype: INTEGER
Column: new_col - Datatype: TEXT


In [405]:
import sqlite3

# Соединение с базой данных SQLite3
conn = sqlite3.connect('data_files/lessons.db')
cursor = conn.cursor()

# Выполнение запроса SELECT для получения уникальных значений из столбца
cursor.execute("SELECT r, list_of_words FROM lessons WHERE r = 1")
results = cursor.fetchall()[0]

print(results)

# Создание списка для хранения результирующих строк
result_list = []

# Разбивка значений и выполнение запросов для каждого числа
r, list_of_words = results
number_list = list_of_words.split(';')
for number in number_list:
    word_index = int(number)
    conn2 = sqlite3.connect('data_files/words.db')
    cursor2 = conn2.cursor()
    cursor2.execute("SELECT * FROM words WHERE word_index = ?", (word_index,))
    word_result = cursor2.fetchone()
    if word_result is not None:
        result_list.append(word_result)

# Вывод результатов
for row in result_list:
    print(row)

# Закрытие соединения с базой данных
conn.close()
conn2.close()

len(result_list)

(1, '6272;3457;2993;6547;7258;1466;680;5270;2440;6551;992;4206;4997;1779;4452;4776;3885;6473;450;4491;6873;1061;4232;3837;4879')
('krapte', 'de', 'shortage', None, None, 'нехватка', 3, 3, 7, 6, 40.0, 6272, 2, 4)
('argument', 'noun, het', 'argument', 'Haar argumenten zijn ongeloofwaardig.', 'Her arguments are implausible.', 'аргумент', 2, 2, 2, 4, 0.0, 3457, 2, 3)
('jouw', 'pron', 'your', 'Jouw paraplu ligt nog hier.', 'Your umbrella is still here.', 'твой', 3, 4, 3, 6, 15.0, 2993, 1, 2)
('achterwerk', 'het', 'ass', None, None, 'жопа', 2, 2, 2, 4, 0.0, 6547, 6, 7)
('broedstoof', 'de', 'incubator', None, None, 'инкубатор', 2, 3, 7, 4, 60.0, 7258, 5, 7)
('schaamte', 'noun, de(f)', 'shame', 'Ik kon wel in de grond kruipen van schaamte.', 'I could crawl into the ground in shame.', 'стыд', 2, 2, 6, 4, 50.0, 1466, 1, 3)
('tekenen', 'verb', 'to draw', 'Het kind tekende een huis en kleurde het daarna in.', 'The child drew a house and then colored it in.', 'рисовать', 6, 8, 6, 12, 15.0, 680, 6, 

25

In [419]:
def get_lesson_words(lesson_number):
    # Соединение с базой данных SQLite3
    conn = sqlite3.connect('data_files/lessons.db')
    cursor = conn.cursor()
    
    conn2 = sqlite3.connect('data_files/words.db')
    cursor2 = conn2.cursor()

    # Выполнение запроса SELECT для получения уникальных значений из столбца
    cursor.execute(f"SELECT r, list_of_words FROM lessons WHERE r = {lesson_number}")
    results = cursor.fetchall()[0]

    # Создание списка для хранения результирующих строк
    result_list = []

    # Разбивка значений и выполнение запросов для каждого числа
    r, list_of_words = results
    number_list = list_of_words.split(';')
    for number in number_list:
        word_index = int(number)
        cursor2.execute("SELECT * FROM words WHERE word_index = ?", (word_index,))
        word_result = cursor2.fetchone()
        if word_result is not None:
            result_list.append(word_result)

    # Закрытие соединения с базой данных
    conn.close()
    conn2.close()
    
    return result_list

In [420]:
get_lesson_words(1)

[('krapte',
  'de',
  'shortage',
  None,
  None,
  'нехватка',
  3,
  3,
  7,
  6,
  40.0,
  6272,
  2,
  4),
 ('argument',
  'noun, het',
  'argument',
  'Haar argumenten zijn ongeloofwaardig.',
  'Her arguments are implausible.',
  'аргумент',
  2,
  2,
  2,
  4,
  0.0,
  3457,
  2,
  3),
 ('jouw',
  'pron',
  'your',
  'Jouw paraplu ligt nog hier.',
  'Your umbrella is still here.',
  'твой',
  3,
  4,
  3,
  6,
  15.0,
  2993,
  1,
  2),
 ('achterwerk', 'het', 'ass', None, None, 'жопа', 2, 2, 2, 4, 0.0, 6547, 6, 7),
 ('broedstoof',
  'de',
  'incubator',
  None,
  None,
  'инкубатор',
  2,
  3,
  7,
  4,
  60.0,
  7258,
  5,
  7),
 ('schaamte',
  'noun, de(f)',
  'shame',
  'Ik kon wel in de grond kruipen van schaamte.',
  'I could crawl into the ground in shame.',
  'стыд',
  2,
  2,
  6,
  4,
  50.0,
  1466,
  1,
  3),
 ('tekenen',
  'verb',
  'to draw',
  'Het kind tekende een huis en kleurde het daarna in.',
  'The child drew a house and then colored it in.',
  'рисовать',
  6

In [418]:
# Соединение с базой данных SQLite3
conn = sqlite3.connect('data_files/lessons.db')
cursor = conn.cursor()

# Выполнение запроса SELECT для получения уникальных значений из столбца
cursor.execute("SELECT DISTINCT r FROM lessons")
results = cursor.fetchall()

unique_values = []

# Вывод уникальных значений в консоль
for row in results:
    unique_values.append(row[0])

unique_values.remove(999)

# Закрытие соединения с базой данных
conn.close()

for l in unique_values:
    result_list = check_all_data(l)
    if len(result_list) != 25:
        print(result_list)

In [ ]:
def get_lesson_words(lesson_number):

    conn = sqlite3.connect('data_files/lessons.db')
    cursor = conn.cursor()
    
    cursor.execute(f"SELECT list_of_words FROM lessons")
    results = cursor.fetchall()
    
    
    
    conn2 = sqlite3.connect('data_files/words.db')
    cursor2 = conn2.cursor()

    
    

    # Создание списка для хранения результирующих строк
    result_list = []

    # Разбивка значений и выполнение запросов для каждого числа
    r, list_of_words = results
    number_list = list_of_words.split(';')
    for number in number_list:
        word_index = int(number)
        cursor2.execute("SELECT * FROM words WHERE word_index = ?", (word_index,))
        word_result = cursor2.fetchone()
        if word_result is not None:
            result_list.append(word_result)

    # Закрытие соединения с базой данных
    conn.close()
    conn2.close()
    
    return result_list

In [430]:
conn = sqlite3.connect('data_files/lessons.db')
cursor = conn.cursor()
cursor.execute(f"SELECT list_of_words FROM lessons")
results = cursor.fetchall()

In [431]:
results

[('6272;3457;2993;6547;7258;1466;680;5270;2440;6551;992;4206;4997;1779;4452;4776;3885;6473;450;4491;6873;1061;4232;3837;4879',),
 ('6086;6109;628;7426;4516;5490;806;955;6790;3056;323;3407;7246;2506;4501;2540;5376;965;2090;2209;6436;6818;7197;4813;5731',),
 ('834;3478;7138;155;3355;3925;1187;3525;1765;6386;6223;7159;3643;7364;4784;2837;7167;6297;761;1485;5012;5555;5300;5421;2928',),
 ('4444;6186;885;2259;7084;1815;2543;6595;6624;6138;800;1566;3454;358;5000;7346;1251;7163;4349;2536;4246;7361;5361;5311;5193',),
 ('3139;3544;320;3416;1025;6963;7408;3509;6030;6390;6280;2919;2641;5000;1532;6864;6209;759;1730;7447;1402;6617;779;7068;5893',),
 ('468;1479;4472;6197;3540;4802;611;3051;6634;5512;3684;2813;7039;7377;4348;1317;4370;7305;4375;784;1495;5614;5405;3642;6676',),
 ('399;887;6037;850;4855;6618;4526;5154;7057;6918;74;937;6188;5727;6847;2405;4378;709;7015;842;1937;4257;1465;465;3624',),
 ('3047;5741;899;2724;7926;4613;2199;6053;7875;4644;1616;669;4957;3659;3552;227;5155;1240;23;3937;7842;50

In [432]:
full_list = []

for list_of_words in results:
    list_of_words = list_of_words[0]
    number_list = list_of_words.split(';')
    for number in number_list:
        number = int(number)
        full_list.append(number)       
full_list = list(set(full_list))

In [433]:
full_list

[8192,
 1,
 2,
 3,
 8,
 8200,
 9,
 8202,
 12,
 8205,
 8204,
 16,
 8210,
 19,
 8211,
 21,
 22,
 23,
 8212,
 8217,
 26,
 8218,
 8219,
 8221,
 30,
 31,
 27,
 8225,
 8226,
 35,
 8227,
 36,
 38,
 34,
 40,
 41,
 8233,
 43,
 8234,
 37,
 8238,
 8239,
 8240,
 48,
 49,
 51,
 52,
 46,
 8245,
 8247,
 55,
 57,
 56,
 60,
 8253,
 61,
 8255,
 8256,
 8257,
 64,
 8259,
 63,
 69,
 8262,
 73,
 74,
 76,
 77,
 8271,
 80,
 8273,
 82,
 8274,
 8276,
 81,
 86,
 8279,
 87,
 8281,
 8278,
 8283,
 8282,
 93,
 91,
 8287,
 92,
 8289,
 97,
 99,
 100,
 101,
 8291,
 103,
 98,
 105,
 106,
 20,
 8300,
 109,
 108,
 8296,
 112,
 8305,
 8306,
 115,
 8310,
 119,
 121,
 8313,
 8315,
 123,
 125,
 25,
 127,
 8319,
 129,
 130,
 8323,
 8324,
 8325,
 134,
 132,
 136,
 8322,
 8330,
 8331,
 140,
 8333,
 141,
 135,
 8336,
 8337,
 145,
 8339,
 147,
 8341,
 151,
 8344,
 155,
 8347,
 157,
 158,
 8354,
 163,
 8356,
 8357,
 8359,
 167,
 169,
 8362,
 8363,
 172,
 8364,
 8366,
 8367,
 173,
 174,
 8369,
 175,
 180,
 181,
 182,
 8375,
 184,
 8

In [434]:
tup = ('uit', 'prep', 'from', 'от', 'Ik kom uit Amsterdam.', "I'm from Amsterdam.", 1, 2, 1, 2, 34.0, 30, 4, 5)

print(tup[-4])

34.0
